In [1]:
## Generate a graph
using StatsBase
using Combinatorics

include("jl/omega.jl")
include("jl/HSBM.jl")
include("jl/hypergraph_louvain.jl")
include("jl/inference.jl");

In [2]:
n = 100
Z = rand(1:2, n)
ϑ = dropdims(ones(1,n) + rand(1,n), dims = 1)

# defining group intensity function Ω
μ = mean(ϑ)

ω(p,α) = (10 .*μ*sum(p))^(-sum(p))*prod(p.^α)^(1/(sum(p)*α))
α0 = 1

kmax = 3

Ω = buildΩ(ω, α0, kmax)
## Sample
H = sampleSBM(Z, ϑ, Ω; α=α0, kmax=kmax, kmin = 1)

hypergraph
  N: Array{Int64}((100,)) [1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  91, 92, 93, 94, 95, 96, 97, 98, 99, 100]
  E: Dict{Int64,Dict}
  D: Array{Int64}((100,)) [2, 1, 1, 1, 1, 2, 3, 0, 2, 1  …  1, 4, 2, 3, 2, 1, 2, 2, 2, 1]


In [4]:
## Run Louvains, Naive, and Faster version

@time Zlou = Naive_HyperLouvain(H,Ω;α=α0)
@time Zlou2 = HyperLouvain(H,kmax,Ω;α=α0)

mlou = modularity(H,Zlou,Ω,α=α0)
mlou2 = modularity(H,Zlou2,Ω,α=α0)

mlou ≈ mlou2
mlou


Louvain Iteration 1
Louvain Iteration 2
Louvain Iteration 3
Louvain Iteration 4
Louvain Iteration 5
Louvain Iteration 6
Louvain Iteration 7
  6.309466 seconds (102.05 M allocations: 2.707 GiB, 31.08% gc time)

Louvain Iteration 1
Louvain Iteration 2
Louvain Iteration 3
Louvain Iteration 4
Louvain Iteration 5
Louvain Iteration 6
Louvain Iteration 7
  0.087979 seconds (875.65 k allocations: 39.177 MiB, 25.43% gc time)


-775.4626549910372122966588349056438200790353221236728131771087646484375

In [7]:
## Try supernode Louvain
# Sanity check: having a singletons initialization is equivalent to naive louvain
# include("jl/hypergraph_louvain.jl")
Z = collect(1:n)
@time Znew, changed = Naive_SuperNodeStep(H,Z,kmax,Ω;α=α0)
mlou3 =  modularity(H,Znew,Ω,α=α0)


Louvain Iteration 1
Louvain Iteration 2
Louvain Iteration 3
Louvain Iteration 4
Louvain Iteration 5
Louvain Iteration 6
Louvain Iteration 7
  7.695958 seconds (123.35 M allocations: 3.272 GiB, 30.59% gc time)


-775.4626549910372122966588349056438200790353221236728131771087646484375

In [10]:
## Now repeatedly call the Naive_SuperNodeStep until no more improvement is possible

@time Z = SuperNodeLouvain(H,kmax,Ω;α=α0)
msuper = modularity(H,Z,Ω,α=α0)

SuperNode Louvain: Phase 1

Louvain Iteration 1
Louvain Iteration 2
Louvain Iteration 3
Louvain Iteration 4
Louvain Iteration 5
Louvain Iteration 6
Louvain Iteration 7
SuperNode Louvain: Phase 2

Louvain Iteration 1
Louvain Iteration 2
Louvain Iteration 3
Louvain Iteration 4
SuperNode Louvain: Phase 3

Louvain Iteration 1
  1.229470 seconds (18.30 M allocations: 623.978 MiB, 28.74% gc time)


-774.0657425925488132277044986580272212250974916969425976276397705078125